In [30]:
import pandas as pd

track_names_mapping_df=pd.read_csv('../../author-profiling-in-symbolic-music/dataframes/track_names_mapping_df.csv')

scores_df=pd.read_csv('../../author-profiling-in-symbolic-music/dataframes/scores_df.csv')

In [31]:
import miditoolkit
import music21
import os
from miditoolkit import MidiFile

def convert_to_type0(midi_file_path, new_filename=None):
    # if new_filename is None:
    #     new_filename = midi_filename.replace('.mid', '_type0.mid')
    
    midi = MidiFile(midi_file_path)
    all_instruments = sum([instrument.notes for instrument in midi.instruments], [])
    
    resulting_midi = midi
    track = miditoolkit.midi.containers.Instrument(program=0, is_drum=False, name='')
    track.notes = sorted(all_instruments, key=lambda x: (x.start, x.pitch))
    resulting_midi.instruments = [track]
    #print(f"Converting {midi_filename} -> {new_filename}")
    
    return resulting_midi

In [34]:
from symusic import TimeUnit,Score
import os

# Function to create a new Score object for each track
def create_scores_for_piano_tracks(scores_df,track_names_mapping_df):
    # Create a list with score objects from the symusic library
    paths=scores_df['scores_paths']
    paths=paths.apply(lambda x: x.replace('../Lieder/','../../Lieder/'))
    
    scores_objects_list = []

    for score in paths:
        scores_objects_list.append(Score(score))

    # Get mapping of clean track names
    track_names_mapping = {}
    for key, value in zip(track_names_mapping_df['track_name_raw'], track_names_mapping_df['track_name_clean']):
        track_names_mapping[key] = value

    full_piano_scores_list = []
    full_piano_scores_paths_list=[]
    paths_type0_list=[]
    # Get piano scores from each song
    for score_path in paths:
        
        score = Score(score_path)
        
        piano_scores_list = []
        piano_scores_paths_list=[]

        for i, track in enumerate(score.tracks):
            
            ## change name of tracks to the clean ones
            track.name=track_names_mapping[track.name] 
            
            
            if track.name == 'piano':
                
                piano_score = Score(ttype=score.ttype)  # Create a new Score object with the same time unit type
                piano_score.ticks_per_quarter = score.ticks_per_quarter  # Set the ticks per quarter note

                # Add global events to the new score
                piano_score.tempos.extend(score.tempos)
                piano_score.time_signatures.extend(score.time_signatures)
                piano_score.key_signatures.extend(score.key_signatures)
                piano_score.lyrics.extend(score.lyrics)
                piano_score.markers.extend(score.markers)

                # Add the track to the new Score object (tracks should be a list)
                piano_score.tracks.append(track)
                
        # Prepare the output path
        output_path = score_path.replace('../Lieder/scores/', '../author-profiling-in-symbolic-music/train data/merged piano scores/')
        output_dir = os.path.dirname(output_path)
        os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

        # Dump each piano new score to a MIDI file
        output_file = f"{output_path}_merged_piano.mid"        
        piano_score.dump_midi(output_file)                
        piano_scores_paths_list.append(output_file)      
        full_piano_scores_paths_list.append(piano_scores_paths_list)

        piano_score_type0=convert_to_type0(output_file)
        # Dump each piano new score to a MIDI file
        output_file_name = f"{output_path}_merged_piano_type0.mid"
        piano_score_type0.dump(output_file_name)
        paths_type0_list.append(output_file_name)

    # Add the new piano scores to the dataframe
    
    scores_df['piano_scores_paths']=full_piano_scores_paths_list
    scores_df['paths_type0_piano']=paths_type0_list

    return scores_df

In [35]:
scores_piano_tracks=create_scores_for_piano_tracks(scores_df=scores_df,track_names_mapping_df=track_names_mapping_df)

In [38]:
scores_piano_tracks.to_csv('../dataframes/piano_merged_scores.csv',index=False)